# 0. Name & Z-ID
Matt Borek

z1951125
"Each of the csv files should have 33 lines (one for each of the 32 artists plus one line for the column names)."
I keep checking, but there are only 31 unique artist names. The CSVs are each 33 lines long, 1 for the columns and 31 for artists, with one following empty line making it 33. Did the instructions misscount?

# 1. Download & Extract Files

In [314]:
import requests
from pathlib import Path
import shutil

course_URL = "https://faculty.cs.niu.edu/~dakoop/cs503-2023sp/a7/"
data_tuples = [(75,79), (80,99), (100,119), (120,199), (200,249), (250,999)]
directory = "artist-data/"
Path(directory).mkdir(exist_ok=True)

for duo in data_tuples:
    fname = f"{duo[0]}_million_to_{duo[1]}_million.zip"
    fpath = Path(directory, fname)

    if not fpath.exists():
        print(f"Downloading \"{fname}\"...")
        connection = requests.get(course_URL + fname)
        with open(fpath, "wb") as new_file:
            new_file.write(connection.content)
        print("Success!")
    else:
        print(f"{fname} already exists.")
    
    data_directory = fname[:-4]
    data_path = Path(directory, data_directory)
    
    if not data_path.exists():
        print(f"Extracting {data_directory} to {directory}...")
        shutil.unpack_archive(str(fpath), directory)
        print("Success!")
    else:
        print(f"{data_directory} already exists.")

Success!
Extracting 75_million_to_79_million to artist-data/...
Success!
Success!
Extracting 80_million_to_99_million to artist-data/...
Success!
Success!
Extracting 100_million_to_119_million to artist-data/...
Success!
Success!
Extracting 120_million_to_199_million to artist-data/...
Success!
Success!
Extracting 200_million_to_249_million to artist-data/...
Success!
Success!
Extracting 250_million_to_999_million to artist-data/...
Success!


# 2. Find Matching Files

In [315]:
npy_files = list(Path(directory).rglob('*.npy'))

# 3. Use Threads to Process Files

In [316]:
import concurrent.futures as cf
import numpy as np
import pandas as pd

def build_dataframe(fpath):
    series_array = np.load(fpath)
    dates = series_array[0]
    views = series_array[1]
    artist = fpath.stem.replace('-', ' ').title()
    
    df = pd.DataFrame({"Date": dates, "Views": views})
    df["Artist"] = artist
    return df
    
with cf.ThreadPoolExecutor() as exec:
    result = exec.map(build_dataframe, npy_files)
df = pd.concat(result)

for month in range(1, 13):
    double_digit = str(month).zfill(2)
    df["Date"] = df["Date"].astype(str)
    month_df = df[df["Date"].str.slice(4, 6) == double_digit]
    year = month_df["Date"].iloc[0][:4]
    month_file = f"{year}{double_digit}.csv.gz"
    
    with open(month_file, 'wb') as newer_file:
        month_df.to_csv(newer_file, compression='gzip', index=False) 